In [1]:
import numpy as np 
from numpy.fft import fft, ifft
import tensorflow as tf
tf.enable_eager_execution()

In [65]:
def _ccorr(a, b):
		a = tf.cast(a, tf.complex64)
		b = tf.cast(b, tf.complex64)
		return tf.real(tf.ifft(tf.conj(tf.fft(a)) * tf.fft(b)))

In [380]:
a = np.random.randn(2,3,5)
b = np.random.randn(2,3,5)

In [381]:
a

array([[[-0.78872213, -0.9822149 ,  1.35201416,  0.64045541,
         -0.39661772],
        [-0.08210258,  0.90494513,  1.13410559, -0.64235496,
          0.6396481 ],
        [-0.34683778,  0.92699383,  0.3485012 , -1.2885408 ,
          1.6141434 ]],

       [[-0.58586308, -1.08653451, -0.77662076, -0.33855393,
         -0.0627085 ],
        [ 0.00852798, -0.88195194,  0.79285947, -0.68993355,
         -0.11300813],
        [ 0.66940726,  1.74844283, -0.59920539,  0.2440645 ,
         -1.63955415]]])

In [382]:
b

array([[[-0.47685239,  0.82988977,  1.5830342 , -0.57162328,
         -1.69599223],
        [-0.17731808, -2.08036528, -0.61797361, -0.2370273 ,
         -1.08961838],
        [ 0.33180697, -1.09511083,  0.25508953,  1.44379657,
          0.37644679]],

       [[-0.86709002, -1.63445292, -1.82052006,  0.27402681,
         -1.41779746],
        [-0.27181144, -0.35085768, -0.50946111,  0.97512246,
         -1.15101522],
        [-0.80570602,  0.24625255, -0.85674162,  1.99657154,
         -0.14367932]]])

In [383]:
_ccorr(a,b)

<tf.Tensor: id=87, shape=(2, 3, 5), dtype=float32, numpy=
array([[[ 2.0078197 , -3.879353  , -3.6146736 ,  1.3756168 ,
          4.168638  ],
        [-3.1136208 , -0.07074246, -2.6163015 , -0.22663462,
         -2.1850116 ],
        [-2.2940958 ,  1.169974  , -0.81410426,  1.7866806 ,
          1.797171  ]],

       [[ 3.6938736 ,  3.257183  ,  2.265978  ,  2.7208562 ,
          3.6412685 ],
        [-0.63950473,  2.0443032 , -1.5497669 ,  1.1075889 ,
          0.19302547],
        [ 1.1274395 , -1.2435449 ,  2.4030862 ,  3.0328622 ,
         -5.1350527 ]]], dtype=float32)>

In [384]:
def np_co(a,b):
    return ifft(np.conj(fft(a)) * fft(b)).real

In [385]:
np_co(a,b)

array([[[ 2.00781995, -3.87935425, -3.6146744 ,  1.37561755,
          4.16863958],
        [-3.11362215, -0.07074224, -2.61630231, -0.22663441,
         -2.1850122 ],
        [-2.29409686,  1.16997405, -0.81410475,  1.78668184,
          1.79717103]],

       [[ 3.6938743 ,  3.25718339,  2.26597767,  2.72085632,
          3.64126888],
        [-0.63950507,  2.0443041 , -1.54976754,  1.1075894 ,
          0.1930255 ],
        [ 1.12743952, -1.2435453 ,  2.40308692,  3.03286358,
         -5.13505412]]])

In [386]:
import torch
def ccorr(a, b):
        #print a
        print a.size()
        k = torch.rfft(a,1,onesided=False)
        t = torch.rfft(b,1,onesided=False)
        k = k *torch.DoubleTensor([1,-1])
        real = (k[:,:,:,0] * t[:,:,:,0]) - (k[:,:,:,1] * t[:,:,:,1])
        imag = (k[:,:,:,0] * t[:,:,:,1]) + (k[:,:,:,1] * t[:,:,:,0])
        a=[]
        print real
        print imag
        for i,j in zip(real,imag):
            for m,n in zip(i,j):
                a.append(m)
                a.append(n)
        a =torch.stack(a,dim=-1)
        t = torch.split(a,2,dim=1)
        t = torch.stack(t,dim=0)
        print t.size()
        v = torch.ifft(t,1)
        #print v.size()
        return v[:,:,0].view(2,-1,5)

In [390]:
import torch
def ccorr(a, b):
        #print a
        print a.size()
        k = torch.rfft(a,1,onesided=False)
        t = torch.rfft(b,1,onesided=False)
        k = k *torch.FloatTensor([1,-1])
        real = (k[:,:,:,0] * t[:,:,:,0]) - (k[:,:,:,1] * t[:,:,:,1])
        imag = (k[:,:,:,0] * t[:,:,:,1]) + (k[:,:,:,1] * t[:,:,:,0])
        #a=[]
        real = real.view(real.shape[0]*real.shape[1],-1)
        imag = imag.view(imag.shape[0]*imag.shape[1],-1)
        a = complex_matrix_multiplication(real,imag)
        #print real
        #print imag
        #for i,j in zip(real,imag):
        #    for m,n in zip(i,j):
        #        a.append(m)
        #        a.append(n)
        #a =torch.stack(a,dim=-1)
        t = torch.split(a,2,dim=1)
        t = torch.stack(t,dim=0)
        print t.size()
        v = torch.ifft(t,1)
        #print v.size()
        return v[:,:,0].view(2,-1,5)

In [391]:
ccorr(torch.Tensor(a),torch.Tensor(b))

torch.Size([2, 3, 5])
torch.Size([6, 5, 2])


tensor([[[ 2.0078, -3.8794, -3.6147,  1.3756,  4.1686],
         [-3.1136, -0.0707, -2.6163, -0.2266, -2.1850],
         [-2.2941,  1.1700, -0.8141,  1.7867,  1.7972]],

        [[ 3.6939,  3.2572,  2.2660,  2.7209,  3.6413],
         [-0.6395,  2.0443, -1.5498,  1.1076,  0.1930],
         [ 1.1274, -1.2435,  2.4031,  3.0329, -5.1351]]])

In [387]:
ccorr(torch.from_numpy(a),torch.from_numpy(b))

torch.Size([2, 3, 5])
tensor([[[ 0.0580,  3.9086,  1.0819,  1.0819,  3.9086],
         [-8.2123, -1.5107, -2.1672, -2.1672, -1.5107],
         [ 1.6456, -2.1640, -4.3940, -4.3940, -2.1640]],

        [[15.5792,  1.7912, -0.3461, -0.3461,  1.7912],
         [ 1.1556,  0.4096, -2.5862, -2.5862,  0.4096],
         [ 0.1848, -5.2414,  7.9676,  7.9676, -5.2414]]], dtype=torch.float64)
tensor([[[  0.0000,  10.5873,  -0.0156,   0.0156, -10.5873],
         [  0.0000,  -0.6062,  -3.5154,   3.5154,   0.6062],
         [  0.0000,   2.1252,  -2.1048,   2.1048,  -2.1252]],

        [[  0.0000,   0.6327,  -0.2069,   0.2069,  -0.6327],
         [  0.0000,  -0.1987,  -3.6155,   3.6155,   0.1987],
         [  0.0000,  -3.3309,  -2.8863,   2.8863,   3.3309]]],
       dtype=torch.float64)
torch.Size([6, 5, 2])


tensor([[[ 2.0078, -3.8794, -3.6147,  1.3756,  4.1686],
         [-3.1136, -0.0707, -2.6163, -0.2266, -2.1850],
         [-2.2941,  1.1700, -0.8141,  1.7867,  1.7972]],

        [[ 3.6939,  3.2572,  2.2660,  2.7209,  3.6413],
         [-0.6395,  2.0443, -1.5498,  1.1076,  0.1930],
         [ 1.1274, -1.2435,  2.4031,  3.0329, -5.1351]]], dtype=torch.float64)

In [223]:
e= np.array([[[ -4.6603,   0.5360,   4.0518,   4.0518,   0.5360],
         [ -0.0087,   7.4644,  -3.3959,  -3.3959,   7.4644],
         [ -0.4402,  -3.1227, -10.2363, -10.2363,  -3.1227]],

        [[  1.3025,   0.4524,  -4.8443,  -4.8443,   0.4524],
         [  8.4571,   0.0527,  -1.5048,  -1.5048,   0.0527],
         [-20.1367,   4.0761,   0.8539,   0.8539,   4.0761]]])

In [224]:
f= np.array([[[-0.0000, -1.7176,  3.7288, -3.7288,  1.7176],
         [ 0.0000, -6.6555, -0.2230,  0.2230,  6.6555],
         [ 0.0000,  4.4860, -2.3758,  2.3758, -4.4860]],

        [[ 0.0000,  2.6392, -3.1743,  3.1743, -2.6392],
         [ 0.0000, -0.2565,  0.2552, -0.2552,  0.2565],
         [-0.0000,  1.5001,  1.9725, -1.9725, -1.5001]]])

In [180]:
a= []
for i,j in zip(e,f):
    for m,n in zip(i,j):
        print m
        print n
        a.append(m)
        a.append(n)

[-4.6603  0.536   4.0518  4.0518  0.536 ]
[-0.     -1.7176  3.7288 -3.7288  1.7176]
[-0.0087  7.4644 -3.3959 -3.3959  7.4644]
[ 0.     -6.6555 -0.223   0.223   6.6555]
[ -0.4402  -3.1227 -10.2363 -10.2363  -3.1227]
[ 0.      4.486  -2.3758  2.3758 -4.486 ]
[ 1.3025  0.4524 -4.8443 -4.8443  0.4524]
[ 0.      2.6392 -3.1743  3.1743 -2.6392]
[ 8.4571  0.0527 -1.5048 -1.5048  0.0527]
[ 0.     -0.2565  0.2552 -0.2552  0.2565]
[-20.1367   4.0761   0.8539   0.8539   4.0761]
[-0.      1.5001  1.9725 -1.9725 -1.5001]


In [183]:
t =np.stack(a,axis=-1)

In [184]:
t

array([[-4.66030e+00, -0.00000e+00, -8.70000e-03,  0.00000e+00,
        -4.40200e-01,  0.00000e+00,  1.30250e+00,  0.00000e+00,
         8.45710e+00,  0.00000e+00, -2.01367e+01, -0.00000e+00],
       [ 5.36000e-01, -1.71760e+00,  7.46440e+00, -6.65550e+00,
        -3.12270e+00,  4.48600e+00,  4.52400e-01,  2.63920e+00,
         5.27000e-02, -2.56500e-01,  4.07610e+00,  1.50010e+00],
       [ 4.05180e+00,  3.72880e+00, -3.39590e+00, -2.23000e-01,
        -1.02363e+01, -2.37580e+00, -4.84430e+00, -3.17430e+00,
        -1.50480e+00,  2.55200e-01,  8.53900e-01,  1.97250e+00],
       [ 4.05180e+00, -3.72880e+00, -3.39590e+00,  2.23000e-01,
        -1.02363e+01,  2.37580e+00, -4.84430e+00,  3.17430e+00,
        -1.50480e+00, -2.55200e-01,  8.53900e-01, -1.97250e+00],
       [ 5.36000e-01,  1.71760e+00,  7.46440e+00,  6.65550e+00,
        -3.12270e+00, -4.48600e+00,  4.52400e-01, -2.63920e+00,
         5.27000e-02,  2.56500e-01,  4.07610e+00, -1.50010e+00]])

In [187]:
t = np.split(t,2,axis=1)

In [188]:
t

[array([[-4.66030e+00, -0.00000e+00, -8.70000e-03,  0.00000e+00,
         -4.40200e-01,  0.00000e+00],
        [ 5.36000e-01, -1.71760e+00,  7.46440e+00, -6.65550e+00,
         -3.12270e+00,  4.48600e+00],
        [ 4.05180e+00,  3.72880e+00, -3.39590e+00, -2.23000e-01,
         -1.02363e+01, -2.37580e+00],
        [ 4.05180e+00, -3.72880e+00, -3.39590e+00,  2.23000e-01,
         -1.02363e+01,  2.37580e+00],
        [ 5.36000e-01,  1.71760e+00,  7.46440e+00,  6.65550e+00,
         -3.12270e+00, -4.48600e+00]]),
 array([[  1.3025,   0.    ,   8.4571,   0.    , -20.1367,  -0.    ],
        [  0.4524,   2.6392,   0.0527,  -0.2565,   4.0761,   1.5001],
        [ -4.8443,  -3.1743,  -1.5048,   0.2552,   0.8539,   1.9725],
        [ -4.8443,   3.1743,  -1.5048,  -0.2552,   0.8539,  -1.9725],
        [  0.4524,  -2.6392,   0.0527,   0.2565,   4.0761,  -1.5001]])]

In [189]:
t = np.stack(t,axis=0)

In [190]:
t

array([[[-4.66030e+00, -0.00000e+00, -8.70000e-03,  0.00000e+00,
         -4.40200e-01,  0.00000e+00],
        [ 5.36000e-01, -1.71760e+00,  7.46440e+00, -6.65550e+00,
         -3.12270e+00,  4.48600e+00],
        [ 4.05180e+00,  3.72880e+00, -3.39590e+00, -2.23000e-01,
         -1.02363e+01, -2.37580e+00],
        [ 4.05180e+00, -3.72880e+00, -3.39590e+00,  2.23000e-01,
         -1.02363e+01,  2.37580e+00],
        [ 5.36000e-01,  1.71760e+00,  7.46440e+00,  6.65550e+00,
         -3.12270e+00, -4.48600e+00]],

       [[ 1.30250e+00,  0.00000e+00,  8.45710e+00,  0.00000e+00,
         -2.01367e+01, -0.00000e+00],
        [ 4.52400e-01,  2.63920e+00,  5.27000e-02, -2.56500e-01,
          4.07610e+00,  1.50010e+00],
        [-4.84430e+00, -3.17430e+00, -1.50480e+00,  2.55200e-01,
          8.53900e-01,  1.97250e+00],
        [-4.84430e+00,  3.17430e+00, -1.50480e+00, -2.55200e-01,
          8.53900e-01, -1.97250e+00],
        [ 4.52400e-01, -2.63920e+00,  5.27000e-02,  2.56500e-01,
      

In [371]:
def complex_matrix_multiplication(a,b):
    combined = torch.zeros((a.shape[0]*2,a.shape[1]))
    a = a.float()
    b = b.float()
    combined[torch.LongTensor(range(0,a.shape[0]*2,2)),:] = a
    combined[torch.LongTensor(range(1,a.shape[0]*2,2)),:] = b
    return combined.transpose_(0,1)

In [372]:
def ccorr2( a, b):
        #print a
        #print a.size()
        k = torch.rfft(a,1,onesided=False)
        t = torch.rfft(b,1,onesided=False)
        k = k *torch.FloatTensor([1,-1])
        real = (k[:,:,0] * t[:,:,0]) - (k[:,:,1] * t[:,:,1])
        imag = (k[:,:,0] * t[:,:,1]) + (k[:,:,1] * t[:,:,0])
        #a=[]
        a = complex_matrix_multiplication(real,imag)
        #for i,j in zip(real,imag):
        #    a.append(i)
        #    a.append(j)
        #print a.type()
        #print a.transpose_(0,1)
        #a =torch.stack(a,dim=-1)
        t = torch.split(a,2,dim=1)
        #print t
        t = torch.stack(t,dim=0)
        v = torch.ifft(t,1)
        #print v.size()
        return v[:,:,0]

In [363]:
def ccorr2( a, b):
        #print a
        #print a.size()
        k = torch.rfft(a,1,onesided=False)
        t = torch.rfft(b,1,onesided=False)
        k = k *torch.FloatTensor([1,-1])
        real = (k[:,:,0] * t[:,:,0]) - (k[:,:,1] * t[:,:,1])
        imag = (k[:,:,0] * t[:,:,1]) + (k[:,:,1] * t[:,:,0])
        a=[]
        #a = complex_matrix_multiplication(real,imag)
        for i,j in zip(real,imag):
            a.append(i)
            a.append(j)
        #print a
        a =torch.stack(a,dim=-1)
        #print a.type()
        #print a
        t = torch.split(a,2,dim=1)
        #print t
        t = torch.stack(t,dim=0)
        v = torch.ifft(t,1)
        #print v.size()
        return v[:,:,0]

In [364]:
c = np.random.randn(2,5)
d = np.random.randn(2,5)

In [365]:
ccorr2(torch.Tensor(c),torch.Tensor(d))

tensor([[-0.9853,  1.1762,  1.2287, -0.6807,  0.0397],
        [ 0.0684,  0.3456, -0.3589, -0.0285, -0.3026]])

In [373]:
ccorr2(torch.FloatTensor(c),torch.FloatTensor(d))

tensor([[-0.9853,  1.1762,  1.2287, -0.6807,  0.0397],
        [ 0.0684,  0.3456, -0.3589, -0.0285, -0.3026]])